# Detecting Topic Merges and Splits in Dynamic Political Conversations

Cláudia Oliveira 

Supervisor - Prof. Dr. Álvaro Figueira

Faculty of Science, University of Porto

Import packages

In [ ]:
import pandas as pd
import re
import string
import emoji
import spacy
#nltk.download('stopwords')
from nltk.corpus import stopwords
import nltk
import sys
from functions import *
from collections import Counter
nlp_ner = spacy.load("en_core_web_sm")
stop_words = set(stopwords.words('english'))
from multiprocessing import Pool, cpu_count

Removing the tweets that aren't in english and concatenate the text from the tweet with the hashtags and urls 

In [2]:
tweets = pd.read_csv("datasets/tweets.csv", encoding="utf-8", low_memory=False)

tweets = tweets[tweets['language'] == 'en'].copy()

tweets['text_combined'] = parallel_apply(tweets, clean_tweet_row)

Eliminate empty lanes

In [12]:
tweets = tweets[tweets['text_combined'].str.strip() != ""].copy()

Check which are the most common used entities so we can try to normalize them 

In [ ]:
all_entities = []

for doc in nlp_ner.pipe(tweets['text_combined'], batch_size=500, n_process=4):
    for ent in doc.ents:
        if ent.label_ in ["PERSON", "ORG"]:
            all_entities.append(ent.text)

entity_counts = Counter(all_entities)
top_entities = entity_counts.most_common(50)

for ent, count in top_entities:
    print(f"{ent}: {count}")

RUN THE PACKAGES BEFORE THIS ONE

In [ ]:
entity_dict = {
    "trump": "Donald Trump",
    "donald trump": "Donald Trump",
    "biden": "Joe Biden",
    "joe biden": "Joe Biden",
    "pelosi": "Nancy Pelosi",
    "nancy pelosi": "Nancy Pelosi",
}

tweets['text_normalized'] = tweets['text_combined'].apply(
    lambda x: normalize_entities_advanced(x, entity_dict, list(entity_dict.keys()))
)